# BERT를 이용한 Chatbot 만들기
> 작성자      
```
* 김성현 (bananaband657@gmail.com)  
김바다 (qkek983@gmail.com)
박상희 (parksanghee0103@gmail.com)  
이정우 (jungwoo.l2.rs@gmail.com)
```
[CC BY-NC-ND](https://creativecommons.org/licenses/by-nc-nd/2.0/kr/)


이번 과제의 목표는 BERT를 사용해 간단한 Chatbot을 어떻게 만들지 고민해 봅니다.

아래 실습 이후, 과제를 위한 챗봇 데이터는 다음의 링크에서 받아서 활용해주세요.   
https://github.com/songys/Chatbot_data   

In [1]:
!pip install transformers

     |████████████████████████████████| 2.1MB 17.9MB/s 
     |████████████████████████████████| 3.3MB 40.0MB/s 
     |████████████████████████████████| 870kB 44.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=dbff1cdcf0f4f6513b1dfc6a3d1b723bd1008efbf5e27b1092025b75f9e51024
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


## Pretrained BERT의  [CLS] token을 이용한 Chatbot.
### 사전 준비물
1. Pretrain된 BERT 모델.
2. 질의응답 Dataset.

### 진행 과정
1. 사용자의 질문(query)를 입력 받는다.
2. query를 pretrained BERT의 입력으로 넣어, query 문장에 해당하는 [CLS] token hidden을 얻는다.
3. 사전에 준비된 질의응답 Dataset에 존재하는 모든 질문들을 pretrained BERT의 입력으로 넣어, 질문들에 해당하는 [CLS] token hidden을 얻는다.
4. query의 [CLS] token hidden과 질문들의 [CLS] token hidden간의 코사인 유사도를 구한다.
5. 가장 높은 코사인 유사도를 가진 질문의 답변을 반환시켜준다.
6. 위 과정 반복.

### Pretrained BERT Load
* 공개된 BERT base Multilingual 버전을 사용한다.

In [2]:
import torch
from transformers import AutoModel, AutoTokenizer

In [3]:
MODEL_NAME = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)
model.parameters

<bound method Module.parameters of BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Drop

### 질의응답 Dataset 예시
* 같은 index가 서로 pair 입니다.

In [4]:
chatbot_Question = ['기차 타고 여행 가고 싶어','꿈이 이루어질까?','내년에는 더 행복해질려고 이렇게 힘든가봅니다', '간만에 휴식 중', '오늘도 힘차게!'] # 질문
chatbot_Answer = ['꿈꾸던 여행이네요.','현실을 꿈처럼 만들어봐요.','더 행복해질 거예요.', '휴식도 필요하죠', '아자아자 화이팅!!'] # 답변
print(chatbot_Question[:])
print(chatbot_Answer[:])

['기차 타고 여행 가고 싶어', '꿈이 이루어질까?', '내년에는 더 행복해질려고 이렇게 힘든가봅니다', '간만에 휴식 중', '오늘도 힘차게!']
['꿈꾸던 여행이네요.', '현실을 꿈처럼 만들어봐요.', '더 행복해질 거예요.', '휴식도 필요하죠', '아자아자 화이팅!!']


### [CLS] token을 얻기 위한 함수

In [5]:
def get_cls_token(sent_A):
    model.eval()
    tokenized_sent = tokenizer(
            sent_A,
            return_tensors="pt",
            truncation=True,
            add_special_tokens=True,
            max_length=128
    )
    with torch.no_grad():# 그라디엔트 계산 비활성화
        outputs = model(    # **tokenized_sent
            input_ids=tokenized_sent['input_ids'],
            attention_mask=tokenized_sent['attention_mask'],
            token_type_ids=tokenized_sent['token_type_ids']
            )
    logits = outputs.last_hidden_state[:,0,:].detach().cpu().numpy()
    return logits

* query [CLS] token hidden 확인

In [6]:
query = '아 여행가고 싶다~'
query_cls_hidden = get_cls_token(query)
print(query_cls_hidden)
print(query_cls_hidden.shape)

[[ 1.10308684e-01 -1.02158815e-01  2.36618251e-01  5.52499592e-02
   3.93724561e-01  2.43569404e-01  1.86364297e-02 -2.02084824e-01
  -9.63338166e-02  3.54374737e-01 -7.72636607e-02  3.26404989e-01
  -1.74165398e-01  6.97088540e-02 -5.34736402e-02 -1.16455413e-01
  -2.62405396e-01  1.22646809e-01  5.06817937e-01 -9.67908725e-02
   1.23077273e-01  7.92987943e-02 -2.98088398e-02  3.16682011e-01
  -1.79102868e-02  2.30731755e-01  3.72240484e-01 -1.27580911e-01
   5.79191685e-01  4.66822088e-01  4.10568893e-01  1.01959750e-01
  -2.56263584e-01  2.49144763e-01  4.32022184e-01  1.58709824e-01
  -1.90560222e+00  1.18138939e-01 -2.43943222e-02 -7.49023333e-02
  -5.16358137e-01  1.96145624e-01  3.76856536e-01  1.58739701e-01
   5.48579514e-01  1.11865151e+00  6.56481624e-01  8.93774033e-02
   1.43516171e+00 -5.34082949e-01  3.55657876e-01 -6.39794528e-01
  -5.98747469e-02 -1.56276703e+00  2.52752602e-01  1.68552980e-01
   3.63083780e-01 -1.66062498e-03  9.51940194e-02 -1.09238461e-01
  -3.28798

* Chatbot 데이터셋의 질문 [CLS] token hidden 확인

In [7]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

dataset_cls_hidden = []
for q in chatbot_Question:
    q_cls = get_cls_token(q)
    dataset_cls_hidden.append(q_cls)
dataset_cls_hidden = np.array(dataset_cls_hidden).squeeze(axis=1)
print(dataset_cls_hidden)   # 데이터셋의 질문에 대한 [CLS] 토큰 벡터
print(dataset_cls_hidden.shape)


[[ 0.20152952  0.0045001   0.1975217  ...  0.5492923   0.02755523
   0.21813363]
 [-0.09731689  0.10722581 -0.23476408 ...  0.36494976  0.45496768
   0.31987897]
 [-0.23625083  0.13388401 -0.2857164  ...  0.53231645  0.334869
   0.30000657]
 [ 0.02712042  0.1217228   0.07334805 ...  0.23747022  0.2893384
  -0.30687705]
 [-0.29900977 -0.09007728  0.10507773 ...  0.37881377  0.38025483
   0.30487245]]
(5, 768)


### 코사인 유사도

In [8]:
cos_sim = cosine_similarity(query_cls_hidden, dataset_cls_hidden)   # 데이터셋의 0번째 질문과 가장 유사하군요!
print(cos_sim)


[[0.85016316 0.7788856  0.73615134 0.77987427 0.7242017 ]]


* Chatbot 데이터 셋중 가장 유사도가 높은 질문 선택 및 답변

In [9]:
top_question = np.argmax(cos_sim)

print('나의 질문: ', query)
print('저장된 답변: ', chatbot_Answer[top_question])

나의 질문:  아 여행가고 싶다~
저장된 답변:  꿈꾸던 여행이네요.
